# Experimental: create a visualization of a topic model with pyLDAvis

In [ ]:
import os.path
from os import path
import sys
import ciso8601
import time
import datetime 
import requests
import json
import csv
import praw
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import gensim
import gensim.corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.test.utils import common_corpus
from gensim.test.utils import datapath
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
nltk.download('wordnet')
import sklearn
import sklearn.model_selection as skmodsel
import sklearn.linear_model as sklinmod
import sklearn.ensemble as skensemble
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import multiprocessing as mp
import pickle
import warnings; warnings.simplefilter('ignore')
import pyLDAvis
import pyLDAvis.sklearn

In [ ]:
def lemmatize_stemming(text):
    '''Function to lemmatize text'''
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [ ]:
def preprocess(text):
    '''Function to pre-process text'''
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
# Experimental: big ugly function
def create_topic_viz(subrname):

    # Subreddit data
    inpath = "./subreddit_data/"
    fname = inpath+"r-"+subrname+"-export.pkl"
    dfraw = pd.read_pickle(fname)

    # Filter out removed posts
    sel = dfraw["selftext"].str.strip() == "[removed]"
    dfraw["selftext"][sel] = ""
    dfraw["title+selftext"] = dfraw["title"]+" "+dfraw["selftext"]
    dfraw["Popularity"] = dfraw["score"]+dfraw["numComms"]

    # Filter out posts with less than 3 words
    n_w = [len(dfraw["title+selftext"][i].split()) for i in range(0,len(dfraw["title+selftext"]))]
    dfraw["n_words"] = n_w
    sel = dfraw["n_words"] > 3
    df = dfraw[sel]    

    outpath = "./nmf_models/"
    outfile = outpath+"nmf_model_prod_r-"+subrname
    nmf_model = pickle.load(open(outfile, 'rb'))

    num_topics = nmf_model.components_.shape[0]
    num_words = nmf_model.components_.shape[1]

    print("r/"+subrname+" Data and Models loaded")

    # Process sentences
    processed_subm = df["title+selftext"].map(preprocess)
    processed_subm_sentences = [' '.join(text) for text in processed_subm]
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', max_features=num_words, smooth_idf=True)
    x_tfidf = tfidf_vectorizer.fit_transform(processed_subm_sentences)
    x_tfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
    x_tfidf_norm = x_tfidf_norm.toarray()
    cc = np.sum(x_tfidf_norm,axis=1)
    isel = x_tfidf_norm > -1.0
    for i in range(0,len(x_tfidf_norm[:,0])):
        cc = np.sum(x_tfidf_norm[i,:])
        if cc == 0.0:
            isel[i,:] = False
    size = int(x_tfidf_norm[isel].shape[0]/num_words)
    x_tfidf_norm1 = np.matrix(x_tfidf_norm[isel].reshape((size,num_words)))
    nmf_model = NMF(n_components=num_topics,init='nndsvd',alpha=0.1)
    y = nmf_model.fit_transform(x_tfidf_norm1)
    features = normalize(y, norm='l1', axis=1)

    # Create visualization and save it
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.sklearn.prepare(nmf_model, x_tfidf_norm1, tfidf_vectorizer)
    pyLDAvis.save_html(viz,"./controversit_app/templates/topic_analysis_r-"+subrname+".html")

    print("r/"+subrname+" Topic Data saved")

In [ ]:
subr_list = [
             "compsci", \
             "politicaldiscussion", \
             "history", \
             "quotes", \
             "changemyview", \
             "parenting" , \
             "confession", \
             "advice"
            ]

for subr in subr_list:
    create_topic_viz(subr)